In [6]:
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/bjn_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/eng_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/ind_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/jav_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/zsm_Latn.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/zho_Hans.dev
# !wget https://github.com/mesolitica/malaysian-dataset/raw/master/translation/flores200-eval/tam_Taml.dev

In [7]:
with open('ind_Latn.dev') as fopen:
    ind = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
ind_, ms_ = [], []
for i in range(len(ind)):
    if len(ind[i]) and len(ms[i]):
        ind_.append(ind[i])
        ms_.append(ms[i])

In [8]:
len(ind_), len(ms_)

(997, 997)

In [9]:
from tqdm import tqdm
import requests
import os
import json

In [5]:
!mkdir google-translate-ind-ms

mkdir: cannot create directory ‘google-translate-ind-ms’: File exists


In [7]:
for i in tqdm(range(len(ms_))):
    filename = os.path.join('google-translate-ind-ms', f'{i}.json')
    
    if os.path.exists(filename):
        continue
    
    headers = {
        'accept': 'application/json',
        'Content-Type': 'application/json',
    }
    
    l = ind_[i]

    json_data = {
        'text': l,
    }

    response = requests.post('http://localhost:8005/translate', headers=headers, json=json_data)
    r = response.json()
    with open(filename, 'w') as fopen:
        json.dump({'text': l, 'r': r}, fopen)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 997/997 [52:42<00:00,  3.17s/it]


In [10]:
from sacrebleu.metrics import BLEU, CHRF, TER

chrf = CHRF(word_order = 2)

In [11]:
predicted, actual = [], []
for i in range(len(ms_)):
    try:
        filename = os.path.join('google-translate-ind-ms', f'{i}.json')
        with open(filename) as fopen:
            d = json.load(fopen)
            
        n = d['r']['result']
        predicted.append(d['r']['result'])
        actual.append(ms_[i])
    except:
        pass

In [12]:
score = chrf.corpus_score(predicted, [actual])
score

chrF2++ = 57.54